# Running Tune experiments with Nevergrad

This example demonstrates the usage of Nevergrad with Ray Tune.

We also combine the search algorithm based on `NevergradSearch` with `AsyncHyperBandScheduler` scheduler to demonstrate Ray Tune's modularity.

Background information:
- [Nevergrad website](https://github.com/facebookresearch/nevergrad)

Necessary requirements:
- `pip install ray[tune]`
- `pip install nevergrad==0.4.3.post7`

In [ ]:
# !pip install ray[tune]
!pip install nevergrad==0.4.3.post7 

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [ ]:
import time

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch

Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune two hyperparameters,
namely `width` and `height`, and `activation`.

In [ ]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 1
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our ``objective`` function takes a Tune ``config``, evaluates the `score` of your experiment in a training loop,
and uses `tune.report` to report the `score` back to Tune.

In [ ]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        tune.report(iterations=step, mean_loss=score)

In [ ]:
ray.init(configure_logging=False)

Now we construct the hyperparameter search space using `ConfigSpace`

Next we define the search algorithm built from `BOHB`, constrained  to a maximum of `4` concurrent trials with a `ConcurrencyLimiter`.

In [ ]:
algo = NevergradSearch(
    optimizer=ng.optimizer.OnePlusOne
    space=space
)
algo = tune.suggest.ConcurrencyLimiter(algo, max_concurrent=4)

Furthermore, we define a `scheduler` to go along with our algorithm to showcase the modularity of Ray Tune.

In [ ]:
scheduler = AsyncHyperBandScheduler() 

The number of samples this Tune run is set to `1000`.
(you can decrease this if it takes too long on your machine).

In [ ]:
num_samples = 1000

In [ ]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Finally, all that's left is to define a search space.

In [ ]:
search_config = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu, tanh"])
}

And run the experiment.

In [ ]:
analysis = tune.run(
    objective,
    search_alg=algo,
    scheduler=scheduler,
    metric="mean_loss",
    mode="min",
    name="nevergrad_exp",
    num_samples=num_samples,
    config=search_config,
)

Here are the hyperparamters found to minimize the mean loss of the defined objective.

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

## Optional: passing the parameter space into the search algorithm

We can also pass the parameter space ourselves

In [ ]:
space = ng.p.Dict(
    width=ng.p.Scalar(lower=0, upper=20),
    height=ng.p.Scalar(lower=-100, upper=100),
    activation=ng.p.Choice(choices=["relu", "tanh"])
)

In [ ]:
algo = NevergradSearch(
    optimizer=ng.optimizer.OnePlusOne
    space=space
)
algo = tune.suggest.ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = AsyncHyperBandScheduler() 

In [ ]:
analysis = tune.run(
    objective,
    search_alg=algo,
    scheduler=scheduler,
    metric="mean_loss",
    mode="min",
    name="nevergrad_exp",
    num_samples=num_samples,
    config=search_config,
)

In [ ]:
ray.shutdown()